In [ ]:
from pymongo import MongoClient
from sklearn.model_selection import train_test_split
from keras.wrappers.scikit_learn import KerasRegressor
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GaussianNoise
from keras.utils import plot_model
from keras.callbacks import TensorBoard
from keras.utils import vis_utils as vizu
from keras.layers import Dropout
from keras import callbacks
from keras import backend
from scipy.integrate import simps
from scipy.stats import f
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import f_regression, mutual_info_regression
def coeff_determination(y_true, y_pred):
    ssres = np.sum(np.square(y_true - y_pred))
    sstot = np.sum(np.square(y_true - np.mean(y_pred)))
    return 1 - ssres / sstot
def REC(y_true , y_pred):   
    Accuracy = []
    Begin_Range = 0
    End_Range = 1.5
    Interval_Size = 0.01
    
    # List of epsilons
    Epsilon = np.arange(Begin_Range , End_Range , Interval_Size)
    
    # Main Loops
   
    print(type(y_true))
    for i in range(len(Epsilon)):
        count = 0.0
        for j in range(len(y_true)):

            y_true[j]=float(y_true[j])
           # print (y_true[j])

         #   print(type(y_true[j]))
    
          #  print(type(y_pred[j]))


            np.linalg.norm(y_true[j]- y_pred[j])

            np.sqrt( np.linalg.norm(y_true[j]) **2 + np.linalg.norm(y_pred[j])**2 ) < Epsilon[i]
            
            if np.linalg.norm(y_true[j] - y_pred[j]) / np.sqrt( np.linalg.norm(y_true[j]) **2 + np.linalg.norm(y_pred[j])**2 ) < Epsilon[i]:
                count = count + 1
        
        Accuracy.append(count/len(y_true))
    
    # Calculating Area Under Curve using Simpson's rule
    AUC = simps(Accuracy , Epsilon ) / End_Range
        
    # returning epsilon , accuracy , area under curve    
    return Epsilon, Accuracy, AUC

def model_evaluate (model, X_train, X_test, y_train, y_test):

    y_pred = model.predict(X_test)
    Deviation, Accuracy, auc = REC(y_test.values, y_pred)
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    #r2 = r2_score(y_test, y_pred)
    r2 = coeff_determination(y_test, y_pred)
    mape = ((y_test - y_pred) / y_test).abs().mean()*100
    
    f_test = np.var(y_test) / np.var(y_pred)
    df1 = len(y_test) - 1
    df2 = len(y_pred) - 1
    p_value = 1 - 2 * abs(0.5 - f.cdf(f_test, df1, df2))
    
    p_stars = ''
    if p_value <= 0.05:
        p_stars = '*'
    if p_value <= 0.01:
        p_stars = '**'
    if p_value <= 0.001:
        p_stars = '***'
    #print(F, p_value)
    
    y_validate = model.predict(X_train)
    training_loss = sqrt(mean_squared_error(y_train, y_validate))
    
    return_dict = {'rmse':rmse, 
                   'mape':mape, 
                   'r2':r2, 
                   'auc':auc,
                   'training_loss':training_loss, 
                   'f_test':f_test,
                   'p':p_stars,
                   'y_pred':y_pred, 
                   'y_validate':y_validate}
    
    return return_dict

def knn_baseline_model():
    
    def root_mean_squared_error(y_true, y_pred):
        return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))
    
    # create model
    model = Sequential()
    #model.add(GaussianNoise(stddev=0.1, input_shape=(14,)))
    model.add(Dense(7, activation='relu', input_shape=(17,)))
    model.add(Dropout(0.5))
    model.add(GaussianNoise(0.1))
    model.add(Dense(24, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss=root_mean_squared_error, optimizer='adam')
    #model.compile(loss='mse', optimizer='adam')
    return model




############


conn = MongoClient('140.115.53.147', 27017)
conn.database_names()
db = conn['NCU_CHKA']


collection = db['chka_final_score']
final_score = pd.DataFrame(list(collection.find()))
#final_score.to_csv('final_score.csv')
collection = db['chka_all_features_by_week']
all_features = pd.DataFrame(list(collection.find()))
#all_features.to_csv('all_features.csv')


all_features = all_features.drop(['_id'], axis=1)
final_score = final_score.drop(['_id'], axis=1)

#preprocessing
mg=all_features.merge(final_score, on='username', how='left')
mg = mg.drop(mg.columns[0], axis=1)

mg = mg.drop(['username'], axis=1)

#mg.to_csv('merge.csv')

data = mg.query("week=='1'")
# 0 會出現 ZeroDivisionError
data =data.query("final_score > '0' ")
data.to_csv('data.csv')
data_y= data['final_score']
data_x = data.drop(['final_score'], axis=1)


#print (data_y)
#print (data_x)
#data_y = data_y[0:15]
#data_x=data_x [0:15]

X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.3)

mlp = KerasRegressor(build_fn=knn_baseline_model, epochs=1000, batch_size=2, verbose=0)
#print(X_train )
#print(y_train)

mlp.fit(X_train, y_train)

#mlp.fit(data_x, data_y)

result_dict = model_evaluate (mlp, X_train, X_test, y_train, y_test)

print(result_dict)


